<a href="https://colab.research.google.com/github/imkunals726/NLP_Disaster_tweets/blob/master/NLP_Disaster_Tweets_LSTM_Glove_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp "drive/My Drive/glove_embedding.zip"  glove.6B.zip 

In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
!pip install kaggle

In [127]:
# from google.colab import files
# files.upload()

In [ ]:

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets list -s nlp-gett
!kaggle datasets download -d misakrug/nlpgettingstarted 

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 40.5MB/s]


In [ ]:
!unzip nlpgettingstarted.zip

Archive:  nlpgettingstarted.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


HELPER Functions

In [ ]:
import keras

Using TensorFlow backend.


In [ ]:
import pandas as pd


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer , TfidfTransformer
from sklearn.preprocessing import StandardScaler

from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer( 'english')

def num_remover( val):
    tokens = val.split()
    nums = [ str(i) for i in range(10)]
    final_tokens = []
    for token in tokens:
        token = token.strip()
        if not any( token.startswith( num ) for num in nums):
            final_tokens.append(token)
    return ' '.join(final_tokens)


def replace_urls(text):
    tokens = text.split()
    
    final_tokens = []
    
    for token in tokens:
        if token.lower().startswith('http'):
            final_tokens.append('url')
        elif token.lower().startswith('@'):
            final_tokens.append('taggeduser')
        else:
            final_tokens.append(token)
    return ' '.join(final_tokens)


def clean_text(df):

    replace_words = [ '&amp' , 'and' , '#' ]

    df['text'] = df['text'].apply(replace_urls)

    for word in replace_words :
        df[ 'text' ] = df[ 'text' ].str.replace( word , '' )

    df[ 'text' ] = df['text' ].apply( lambda txt : ' '.join( stemmer.stem(lemmatizer.lemmatize( word ) ) for word in txt.split( ' ') ) )

    df['keyword'] = df['keyword'].fillna('').str.replace('%20' , ' ')
    df[ 'text' ] = df.apply( lambda row : str( row[ 'text' ] ) + ' ' + str(row[ 'keyword' ]) if row[ 'keyword' ] else row[ 'text' ] , axis = 1)

    df['text'] = df['text'].apply(num_remover)
    
    return df

In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Lets Clean the text

In [ ]:

# train_df = clean_text(train_df)
train_df['text_sequences'] = train_df['text'].apply(keras.preprocessing.text.text_to_word_sequence)
train_df['text_sequences'].head()

0    [our, deeds, are, the, reason, of, this, earth...
1        [forest, fire, near, la, ronge, sask, canada]
2    [all, residents, asked, to, 'shelter, in, plac...
3    [13, 000, people, receive, wildfires, evacuati...
4    [just, got, sent, this, photo, from, ruby, ala...
Name: text_sequences, dtype: object

In [ ]:
embeddings_index = {}
f = open('glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


In [ ]:
len(embeddings_index)

400000

In [ ]:
word_index = {key:idx for idx,key in enumerate(embeddings_index.keys())}

In [ ]:
train_df = pd.read_csv('train.csv')

train_df['text_sequences'] = train_df['text'].apply(keras.preprocessing.text.text_to_word_sequence)

def get_sequence(text_sequence):
  sequence = []
  for word in text_sequence:
    if word in word_index:
      sequence.append(word_index[word])
    else:
      sequence.append(len(word_index))
  return sequence
train_df['sequence'] = train_df['text_sequences'].apply(get_sequence)

In [ ]:

train_df[['text_sequences' , 'sequence']].head()

,text_sequences,sequence
0,"[our, deeds, are, the, reason, of, this, earth...","[162, 13091, 32, 0, 1247, 3, 37, 3117, 107, 14..."
1,"[forest, fire, near, la, ronge, sask, canada]","[2061, 484, 355, 1047, 159475, 270969, 774]"
2,"[all, residents, asked, to, 'shelter, in, plac...","[64, 1048, 476, 4, 400000, 6, 400000, 32, 134,..."
3,"[13, 000, people, receive, wildfires, evacuati...","[676, 14077, 69, 1653, 18161, 6904, 1949, 6, 628]"
4,"[just, got, sent, this, photo, from, ruby, ala...","[120, 405, 688, 37, 3119, 25, 12650, 4604, 19,..."


In [ ]:
from sklearn.model_selection  import train_test_split
X_train, X_val , y_train, y_val = train_test_split(train_df[['text']] , train_df['target'])

In [ ]:

import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = len(word_index)+1
embedding_dim = 200
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
def prepare_data(sequences_to_pad):
  # word_index = tokenizer.word_index
  # training_sequences = tokenizer.texts_to_sequences(sentences_to_convert)
  sequences_to_pad = sequences_to_pad.apply(keras.preprocessing.text.text_to_word_sequence)
  sequences_to_pad = sequences_to_pad.apply(get_sequence)
  training_padded = pad_sequences(sequences_to_pad, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  return training_padded

In [ ]:
training_text = X_train['text']


testing_text = X_val['text']

training_padded = prepare_data(training_text)
testing_padded = prepare_data(testing_text)


In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], input_length=max_length, trainable=False),

      tf.keras.layers.LSTM(units = 128, return_sequences=True),
      tf.keras.layers.Dropout(0.5),      
      tf.keras.layers.LSTM(units = 128, return_sequences=True),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.LSTM(units = 128, return_sequences=True),
      tf.keras.layers.Dropout(0.5),      
      tf.keras.layers.GlobalAveragePooling1D(),
      tf.keras.layers.Dense(128,activation='relu'),
      tf.keras.layers.Dropout(0.5),      
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model

In [ ]:
model = create_model()
model.summary()


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 100, 200)          80000200  
_________________________________________________________________
lstm_33 (LSTM)               (None, 100, 128)          168448    
_________________________________________________________________
dropout_39 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 100, 128)          131584    
_________________________________________________________________
dropout_40 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 100, 128)          131584    
_________________________________________________________________
dropout_41 (Dropout)         (None, 100, 128)        

In [ ]:
num_epochs = 2
history = model.fit(training_padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_val), verbose=2)

Epoch 1/2


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
xtf.keras.backend.set_value(model.optimizer.lr,0.01)
num_epochs = 10
history = model.fit(training_padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_val), verbose=2)

Epoch 1/10
179/179 - 4s - loss: 0.6836 - accuracy: 0.5710 - val_loss: 0.6848 - val_accuracy: 0.5683
Epoch 2/10
179/179 - 4s - loss: 0.6834 - accuracy: 0.5710 - val_loss: 0.6844 - val_accuracy: 0.5683
Epoch 3/10
179/179 - 4s - loss: 0.6836 - accuracy: 0.5710 - val_loss: 0.6841 - val_accuracy: 0.5683
Epoch 4/10
179/179 - 4s - loss: 0.6835 - accuracy: 0.5710 - val_loss: 0.6838 - val_accuracy: 0.5683
Epoch 5/10
179/179 - 4s - loss: 0.6835 - accuracy: 0.5710 - val_loss: 0.6844 - val_accuracy: 0.5683
Epoch 6/10
179/179 - 4s - loss: 0.6832 - accuracy: 0.5710 - val_loss: 0.6845 - val_accuracy: 0.5683
Epoch 7/10
179/179 - 4s - loss: 0.6834 - accuracy: 0.5710 - val_loss: 0.6841 - val_accuracy: 0.5683
Epoch 8/10
179/179 - 4s - loss: 0.6835 - accuracy: 0.5710 - val_loss: 0.6838 - val_accuracy: 0.5683
Epoch 9/10
179/179 - 4s - loss: 0.6832 - accuracy: 0.5710 - val_loss: 0.6838 - val_accuracy: 0.5683
Epoch 10/10
179/179 - 4s - loss: 0.6833 - accuracy: 0.5710 - val_loss: 0.6841 - val_accuracy: 0.5683

In [ ]:
test_df = pd.read_csv('test.csv')

In [ ]:
# test_df = clean_text(test_df)

In [ ]:
train_df = clean_text(train_df)

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_df['text'])

In [ ]:
training_padded = prepare_data(train_df['text'] )

In [ ]:
testing_padded = prepare_data(test_df['text'] )

In [ ]:
model = create_model()

In [ ]:
num_epochs = 10
history = model.fit(training_padded, train_df['target'], epochs=num_epochs, verbose=2)

Epoch 1/10
238/238 - 11s - loss: 0.5884 - accuracy: 0.7081
Epoch 2/10
238/238 - 11s - loss: 0.5251 - accuracy: 0.7692
Epoch 3/10
238/238 - 10s - loss: 0.4906 - accuracy: 0.7872
Epoch 4/10
238/238 - 10s - loss: 0.4344 - accuracy: 0.8124
Epoch 5/10
238/238 - 10s - loss: 0.4138 - accuracy: 0.8236
Epoch 6/10
238/238 - 10s - loss: 0.3986 - accuracy: 0.8333
Epoch 7/10
238/238 - 10s - loss: 0.3796 - accuracy: 0.8401
Epoch 8/10
238/238 - 10s - loss: 0.3651 - accuracy: 0.8521
Epoch 9/10
238/238 - 9s - loss: 0.3436 - accuracy: 0.8643
Epoch 10/10
238/238 - 9s - loss: 0.3208 - accuracy: 0.8719


In [ ]:
history1 = model.fit(training_padded, train_df['target'], epochs=5, verbose=2)

Epoch 1/5
238/238 - 10s - loss: 0.2947 - accuracy: 0.8841
Epoch 2/5
238/238 - 10s - loss: 0.2671 - accuracy: 0.8990
Epoch 3/5
238/238 - 10s - loss: 0.2515 - accuracy: 0.9050
Epoch 4/5
238/238 - 10s - loss: 0.2173 - accuracy: 0.9232
Epoch 5/5
238/238 - 10s - loss: 0.1937 - accuracy: 0.9308


In [ ]:
results = model.predict(testing_padded)

In [ ]:
 results = (results > 0.5).astype(int)

In [ ]:
results

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [ ]:
test_df.head(1)['text'].tolist()[0]

'Just happened a terrible car crash'

In [ ]:
test_df['target'] = results

In [ ]:
test_df.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
submission = test_df[['id' , 'target']]

In [ ]:
submission.to_csv('using_embeddings_and_lstm_1.csv', index=False)